In [10]:
import pickle
import numpy as np
import keras
from keras import models
from keras import layers
import cv2
import matplotlib.pyplot as plt
import gradio as gr
import warnings
warnings.filterwarnings("ignore")

### Cargamos ambos modelos (Regresión y Redes Neuronales)

In [11]:
modelo = pickle.load(open("modelo_xgboost.pkl", 'rb'))

In [12]:
red = keras.models.load_model("modelo final save")

### Función para el modelo de regresión (XGBoost)

In [13]:
def usuario(registro):
    marca = ['Cartier', 'Casio', 'Hamilton', 'Hublot',
       'Omega', 'Patek Philippe', 'Richard Mille',
       'Rolex', 'Seiko', 'Tissot']
    materialCierre = ['Acero', 'Acero y oro',
           'Aluminio', 'Bronce',
           'Carbono', 'Cerámica',
           'Oro amarillo', 'Oro blanco',
           'Oro rojo', 'Oro rosa',
           'Paladio', 'Plata',
           'Platino', 'Plástico',
           'Titanio', 'Tántalo']

    contenidoEntrega = [ 'Con documentos originales, sin estuche original',
           'Con estuche original, con documentos originales',
           'Con estuche original, sin documentos originales',
           'Sin estuche original, sin documentos originales',]
    tipoCalibre = ['Automático', 'Cuarzo',
           'Cuerda manual']
    materialCaja = ['Acero',
           'Acero y oro', 'Aluminio',
           'Bronce', 'Carbono',
           'Cerámica', 'Oro amarillo',
           'Oro blanco', 'Oro rojo',
           'Oro rosa', 'Paladio',
           'Plata', 'Platino',
           'Plástico', 'Titanio',
           'Tántalo']
    estado = ['Bueno                     (Señales ligeras hasta notables de uso o arañazos)',
           'Defectuoso                     (Señales evidentes de uso o arañazos)',
           'Muy bueno                     (Usado, sin o casi sin señales de uso)',
           'Nuevo                     (Totalmente nuevo (de fabrica) sin señales de uso)',
           'Satisfactorio                     (Señales notables de uso o arañazos)',
           'Sin usar                     (Nuevo de un stock antiguo, sin señales de uso)']
    materialPulsera = ['Acero', 'Acero y oro',
           'Aluminio', 'Carbono',
           'Caucho', 'Cerámica',
           'Oro amarillo',
           'Oro blanco', 'Oro rojo',
           'Oro rosa', 'Piel',
           'Piel de aligátor',
           'Piel de avestruz',
           'Piel de lagarto',
           'Piel de serpiente',
           'Piel de tiburon',
           'Piel de vaca', 'Plata',
           'Platino', 'Plástico',
           'Satén', 'Silicona',
           'Textil', 'Titanio',]
    esfera = ['Amarillo', 'Azul', 'Blanco', 'Bronce',
           'Burdeos', 'Champán', 'Gris', 'Madreperla',
           'Marrón', 'Naranja', 'Negro', 'Oro',
           'Oro (macizo)', 'Plata', 'Plata (maciza)',
           'Rojo', 'Rosa', 'Transparente', 'Verde',
           'Violeta']
    
    listOfLists=[marca, materialCierre, contenidoEntrega, tipoCalibre, materialCaja, estado, materialPulsera, esfera]
    
    copyRegistro = registro.copy()
    finalArray = np.array(copyRegistro[0][0:4]) # Inicializo el array con los 4 primeros elementos (numéricos)
    deleteC = np.delete(copyRegistro,0)
    deleteC = np.delete(deleteC,0)
    deleteC = np.delete(deleteC,0)
    deleteC = np.delete(deleteC,0) # deleteC es el nuevo array solo con los categóricos
    for i in range(len(listOfLists)):
        ar = np.zeros(len(listOfLists[i]))
        if deleteC[i] in listOfLists[i]: # Añado esta condición para que busque si existe y aplique
            index = listOfLists[i].index(deleteC[i])
    
            ar[index] = 1
        
        finalArray = np.concatenate((finalArray, ar))
        

    finalArray = finalArray.reshape(1,103)
    
    precio= modelo.predict(finalArray)
    
    return precio

### Función para el modelo de Redes Neuronales

In [14]:
def imagen(img):
    img = np.array(img)
    ok=cv2.resize(img,(200,200))
    gray=cv2.cvtColor(ok,cv2.COLOR_BGR2GRAY)
    gray_3d = np.expand_dims(gray, axis=2)
    gray_3d = np.repeat(gray_3d,3, axis=2)
    relojes = ["Cartier Santos","Hublot Big Bang","Omega SpeedMaster","Patek Philippe Nautilus","Rolex Submariner"]
    predict=red.predict(gray_3d.reshape((1,200,200,3)))
    
    abc = np.argmax(predict)  
    if abc == 0:
        return "Su reloj es un Cartier Santos"
    elif abc == 1:
        return "Su reloj es un Hublot Big Bang"
    elif abc == 2:
        return "Su reloj es un Omega SpeedMaster"
    elif abc ==3:
        return "Su reloj es un Patek Philippe Nautilus"
    elif abc ==4:
        return "Su reloj es un Rolex Submariner"
    

### DEMO DE NUESTRO DESPLIEGUE

In [21]:
def greet(Marca,Cierre,Calibre,Entrega,Caja,Estado,Pulsera,Esfera,Perímetro,Reserva_de_marcha,Joyas,Resistencia_al_agua):
    prueba = np.array([[Perímetro,Joyas,Reserva_de_marcha,Resistencia_al_agua,Marca,Cierre,Entrega,Calibre,Caja,Estado,Pulsera,Esfera]], dtype = object) 
    precio = str(int(usuario(prueba)[0].round(0)))
    return "Su reloj tiene un precio estimado de " + precio + "€"


valor = gr.Interface(greet,[gr.Dropdown(['Cartier', 'Casio', 'Hamilton', 'Hublot',
       'Omega', 'Patek Philippe', 'Richard Mille',
       'Rolex', 'Seiko', 'Tissot']),gr.Dropdown(['Acero', 'Acero y oro',
        'Aluminio', 'Bronce',
           'Carbono', 'Cerámica',
           'Oro amarillo', 'Oro blanco',
           'Oro rojo', 'Oro rosa',
           'Paladio', 'Plata',
           'Platino', 'Plástico',
           'Titanio', 'Tántalo']),gr.Radio(['Automático', 'Cuarzo',
           'Cuerda manual']),gr.Dropdown([ 'Con documentos originales, sin estuche original',
           'Con estuche original, con documentos originales',
           'Con estuche original, sin documentos originales',
           'Sin estuche original, sin documentos originales',]),gr.Dropdown(['Acero',
           'Acero y oro', 'Aluminio',
           'Bronce', 'Carbono',
           'Cerámica', 'Oro amarillo',
           'Oro blanco', 'Oro rojo',
           'Oro rosa', 'Paladio',
           'Plata', 'Platino',
           'Plástico', 'Titanio',
           'Tántalo']),gr.Dropdown(['Bueno                     (Señales ligeras hasta notables de uso o arañazos)',
           'Defectuoso                     (Señales evidentes de uso o arañazos)',
           'Muy bueno                     (Usado, sin o casi sin señales de uso)',
           'Nuevo                     (Totalmente nuevo (de fabrica) sin señales de uso)',
           'Satisfactorio                     (Señales notables de uso o arañazos)',
           'Sin usar                     (Nuevo de un stock antiguo, sin señales de uso)']), gr.Dropdown(['Acero', 'Acero y oro',
           'Aluminio', 'Carbono',
           'Caucho', 'Cerámica',
           'Oro amarillo',
           'Oro blanco', 'Oro rojo',
           'Oro rosa', 'Piel',
           'Piel de aligátor',
           'Piel de avestruz',
           'Piel de lagarto',
           'Piel de serpiente',
           'Piel de tiburon',
           'Piel de vaca', 'Plata',
           'Platino', 'Plástico',
           'Satén', 'Silicona',
           'Textil', 'Titanio']),gr.Dropdown(['Amarillo', 'Azul', 'Blanco', 'Bronce',
           'Burdeos', 'Champán', 'Gris', 'Madreperla',
           'Marrón', 'Naranja', 'Negro', 'Oro',
           'Oro (macizo)', 'Plata', 'Plata (maciza)',
           'Rojo', 'Rosa', 'Transparente', 'Verde',
           'Violeta']),gr.Slider(80,300,value=190), gr.Slider(0,400,value=200,label="Reserva de marcha"), gr.Slider(0,120,value=60),
                           gr.Slider(0,150,value=75,label="Resistencia al agua")], outputs =gr.Label(label="Esta será su predicción"), 
   
                    css="body {background-image: url('https://i.pinimg.com/originals/9b/ff/38/9bff3810a2423dfb55eb15e49d043e72.jpg')}",
                    title="TagMyWatch ⌚💶")

clasificador = gr.Interface(imagen,gr.Image(type="pil"), outputs =gr.Label(label="Este será su reloj"),
                     css="body {background-image: url('https://i.pinimg.com/originals/9b/ff/38/9bff3810a2423dfb55eb15e49d043e72.jpg')}",
                           title="TagMyWatch ⌚💶")

demo=gr.TabbedInterface([clasificador,valor],['Conoce tu modelo','Valora tu reloj'])


In [22]:
demo.launch()

Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 209ms/step
